<a href="https://colab.research.google.com/github/pnjha/NLP-Assignments/blob/master/nlp12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive


In [0]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import copy
import math
import tensorflow as tf
from keras.optimizers import SGD
from keras.initializers import RandomUniform
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import RepeatVector, Bidirectional, Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout, Activation
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

In [5]:
X = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.en', names=['src'])
Y = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.hi', names=['dest'])
lines = pd.concat([X[:3000],Y[:3000]], axis=1)
print(len(lines))
lines = shuffle(lines)
lines.sample(10)

3000


,src,dest
1982,Many methods of cancer detection like mammogra...,अभी तक कैंसर जाँच की कई विधियाँ जैसे मैमोग्राफ...
1137,The health of family depends on health of wome...,महिलाओं के स्वास्थ्य पर ही परिवार का स्वास्थ्य...
1539,The food eaten does not get digested .,खाया-पिया पचता नहीं है ।
2082,In FSH test one can be saved from the diseases...,एफएसएच टेस्ट से मेनोपाज के समय या बाद में होने...
1656,Full of protein and vitamins where corn provid...,प्रोटीन और विटामिनों से भरपूर मक्का जहाँ शरीर ...
1550,Therefore there is deficiency of water in body .,इसलिए शरीर में पानी की कमी हो जाती है ।
2749,Genetic factor may also be a cause for blood p...,जेनीटिक फैक्टर भी रक्‍तचाप के लिए एक कारण हो स...
2825,On the other hand treatment of prostrate throu...,दूसरी ओर लेजर किरणों के प्रयोग द्वारा प्रोस्टे...
2879,"To reduce the chances of allergy , asthma in f...","भविष्य में बच्चों को एलर्जी , दमे की संभावना क..."
403,About 25 to 50 percent of rubella contraction ...,लगभग 25 से 50 प्रतिशत रूबेला संक्रमण का पता नह...


In [0]:
def process_data(data,append_char):
  data = data.apply(lambda x: x.lower())
  data = data.apply(lambda x: x.strip())
  data = data.apply(lambda x: re.sub("'", '', x))
  exclude = set(string.punctuation) # Set of all special characters
  data = data.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
  remove_digits = str.maketrans('', '', digits)
  data = data.apply(lambda x: x.translate(remove_digits))
  if append_char:
    data = data.apply(lambda x : '<SOS> '+ x + ' <EOS>')
  return data

In [7]:
lines.src = process_data(lines.src,False)
lines.dest = process_data(lines.dest,True)
lines.sample(10)

,src,dest
758,anything altered on eyes has direct impact on ...,<SOS> आँखों पर हुआ कुछ भी परिवर्तन सीधा किडनी ...
2333,which has spread in other parts of the body as...,<SOS> जो शरीर के अन्य भागों में भी फैल चुका है...
2055,those women who are unmarried or then those wh...,<SOS> वे महिलाएँ जो विवाहित नहीं हैं या फिर श...
607,yellow fever spread the worst in in the unite...,<SOS> पीतज्वर संयुक्त राज्य मे सबसे बुरी तरह ...
1728,but due to these medicines the symptoms of bon...,<SOS> पर इन दवाओं से हड्डियों में कमजोरी और मी...
1744,in bacterial dysentery there is repeated disch...,<SOS> दंडाणुज पेचिश में रक्‍त और श्‍लेष्मा से ...
1433,for the parents who have children with diabete...,<SOS> डायबिटीज से ग्रस्त बच्चों के मातापिताओं ...
992,heart patient should use less salt chilli and...,<SOS> हृदय रोगी को नमक मिर्च तथा तलेभुने भोज...
1954,it can easily be estimated by this that our le...,<SOS> इससे यह अनुमान सहज ही लगाया जा सकता है क...
2086,one s cholesterol must be checked regularly af...,<SOS> साल के होते ही अपने कोलेस्टरोल की नियमि...


In [0]:
def get_vocab_list(data):
  vocab = []
  for line in data:
    temp = line.split()
    for item in temp:
      if item not in vocab:
        vocab.append(item)
  return vocab

In [0]:
src_vocab = get_vocab_list(lines.src)
dest_vocab = get_vocab_list(lines.dest)

In [18]:
print(src_vocab[:10])
print(dest_vocab[:10]) 

['pulse', 'polio', 'mission', 'is', 'actually', 'a', 'team', 'work', 'in', 'which']
['<SOS>', 'पल्स', 'पोलियो', 'अभियान', 'वास्तव', 'में', 'एक', 'टीम', 'वर्क', 'है']


In [0]:
# Max Length of source sequence
def get_max_sentence_size(data):
  max_len = 0
  for line in data:
    temp = line.split()
    max_len = max(max_len, len(temp))
  return max_len

In [20]:
src_max_len = get_max_sentence_size(lines.src)
dest_max_len = get_max_sentence_size(lines.dest)
src_max_len,dest_max_len

(84, 86)

In [21]:
src_vocab_len = len(src_vocab)
dest_vocab_len = len(dest_vocab) + 1
src_vocab_len, dest_vocab_len

(4711, 6289)

In [0]:
def get_word_to_index_dict(vocab):
  word_to_index = {}
  for i, word in enumerate(vocab):
    word_to_index[word] = i+1
  return word_to_index

In [0]:
src_word_to_index = get_word_to_index_dict(src_vocab)
dest_word_to_index = get_word_to_index_dict(dest_vocab)

In [24]:
print(src_word_to_index)
print(dest_word_to_index)

{'pulse': 1, 'polio': 2, 'mission': 3, 'is': 4, 'actually': 5, 'a': 6, 'team': 7, 'work': 8, 'in': 9, 'which': 10, 'there': 11, 'equal': 12, 'contribution': 13, 'of': 14, 'each': 15, 'member': 16, 'take': 17, 'half': 18, 'gram': 19, 'powder': 20, 'with': 21, 'rice': 22, 'water': 23, 'the': 24, 'morning': 25, 'if': 26, 'you': 27, 'two': 28, 'simple': 29, 'roller': 30, 'pumps': 31, 'place': 32, 'halves': 33, 'heart': 34, 'auricle': 35, 'and': 36, 'ventricle': 37, 'then': 38, 'can': 39, 'do': 40, 'entire': 41, 'task': 42, 'for': 43, 'this': 44, 'order': 45, 'to': 46, 'ascertain': 47, 'abundance': 48, 'substances': 49, 'lrb': 50, 'antibody': 51, 'rrb': 52, 'fight': 53, 'against': 54, 'human': 55, 'body': 56, 'dose': 57, 'drops': 58, 'sobin': 59, 'vaccine': 60, 'given': 61, 'through': 62, 'mouth': 63, 'salk': 64, 'injection': 65, 'under': 66, 'immunity': 67, 'stone': 68, 'bladder': 69, 'forms': 70, 'more': 71, 'children': 72, 'old': 73, 'people': 74, 'while': 75, 'adults': 76, 'form': 77, '

In [0]:
def get_index_to_word_dict(vocab):
  index_to_word = {}
  for i, word in enumerate(vocab):
    index_to_word[i+1] = word
  return index_to_word  

In [0]:
src_index_to_word = get_index_to_word_dict(src_vocab)
dest_index_to_word = get_index_to_word_dict(dest_vocab)

In [27]:
print(src_index_to_word)
print(dest_index_to_word)

{1: 'pulse', 2: 'polio', 3: 'mission', 4: 'is', 5: 'actually', 6: 'a', 7: 'team', 8: 'work', 9: 'in', 10: 'which', 11: 'there', 12: 'equal', 13: 'contribution', 14: 'of', 15: 'each', 16: 'member', 17: 'take', 18: 'half', 19: 'gram', 20: 'powder', 21: 'with', 22: 'rice', 23: 'water', 24: 'the', 25: 'morning', 26: 'if', 27: 'you', 28: 'two', 29: 'simple', 30: 'roller', 31: 'pumps', 32: 'place', 33: 'halves', 34: 'heart', 35: 'auricle', 36: 'and', 37: 'ventricle', 38: 'then', 39: 'can', 40: 'do', 41: 'entire', 42: 'task', 43: 'for', 44: 'this', 45: 'order', 46: 'to', 47: 'ascertain', 48: 'abundance', 49: 'substances', 50: 'lrb', 51: 'antibody', 52: 'rrb', 53: 'fight', 54: 'against', 55: 'human', 56: 'body', 57: 'dose', 58: 'drops', 59: 'sobin', 60: 'vaccine', 61: 'given', 62: 'through', 63: 'mouth', 64: 'salk', 65: 'injection', 66: 'under', 67: 'immunity', 68: 'stone', 69: 'bladder', 70: 'forms', 71: 'more', 72: 'children', 73: 'old', 74: 'people', 75: 'while', 76: 'adults', 77: 'form', 7

In [0]:
def reverse_train_sentences(data):
  l = data.values.tolist()
  t = []
  for item in l:
    p = item.strip().split()
    p.reverse()
    k = []
    for i in p:
      i = i.strip()
      if len(i) > 0:
        k.append(i)
    p = k
    p = " ".join(p)
    p = p.strip()
    t.append(p)
  df = pd.DataFrame({'data':t})
  return df.data  

In [29]:
# Train - Test Split
X, y = lines.src, lines.dest
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train_normal = copy.deepcopy(X_train)
X_train = reverse_train_sentences(X_train)
print(X_train.shape, X_test.shape)
X_train.sample(10)

(2700,) (300,)


1865    head entire the in pain have percent and head ...
750     pain waist is there so tears disc less being w...
1559                     same both of methods the are but
763     it against available vaccine any there is nor ...
1295    process digestive in occurs disorder then fast...
339     habits proper and style life balanced adopting...
1765                             hour an for frisbee play
1403           sugar without tea ice or coffee black take
1677      pneumonia resemble sars of symptoms initial the
2207    tuberculosis of because homes from away thrown...
Name: data, dtype: object

In [0]:
def generate_batch(X = X_train, y = y_train):
  batch_size = len(X)
  encoder_input_data = np.zeros((batch_size, src_max_len),dtype='float32')
  decoder_target_data = np.zeros((batch_size, dest_max_len, dest_vocab_len),dtype='float32')
  decoder_data = np.zeros((batch_size, dest_max_len),dtype='float32')

  for i, (input_text, target_text) in enumerate(zip(X,y)):
      for t, word in enumerate(input_text.split()):
          encoder_input_data[i, t] = src_word_to_index[word]
      for t, word in enumerate(target_text.split()):
          decoder_target_data[i, t-1, dest_word_to_index[word]] = 1
          decoder_data[i,t] = dest_word_to_index[word]
  return encoder_input_data, decoder_target_data, decoder_data

In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [0]:
latent_dim = 300

In [32]:
model = Sequential()

# Creating encoder network
model.add(Embedding(input_dim = src_vocab_len, output_dim = 300, input_length = src_max_len, mask_zero = True))
model.add(LSTM(units = latent_dim,return_sequences=True,kernel_initializer=RandomUniform(minval=-0.08, maxval=0.08, seed=None)))
model.add(LSTM(units = latent_dim,kernel_initializer=RandomUniform(minval=-0.08, maxval=0.08, seed=None)))

# Creating decoder network
model.add(RepeatVector(dest_max_len))
model.add(LSTM(units = latent_dim, return_sequences=True,kernel_initializer=RandomUniform(minval=-0.08, maxval=0.08, seed=None)))
model.add(LSTM(units = latent_dim, return_sequences=True,kernel_initializer=RandomUniform(minval=-0.08, maxval=0.08, seed=None)))
model.add(Dense(dest_vocab_len,activation='softmax'))
# model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer=SGD(lr=0.7),metrics=['accuracy'])
print(model.summary())




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 84, 300)           1413300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 84, 300)           721200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 86, 300)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 86, 300)           721200    
_________________________________________________________________
lstm_4 (LSTM)                (None, 86, 300)           721200    
_____________________

In [0]:
X,y, yp = generate_batch(X_train, y_train)
xtest, ytest, ytestp = generate_batch(X_test, y_test)

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
epochs = 30
batch_size = 64

In [37]:
model.fit(X,y,
          epochs=epochs,
          batch_size=batch_size,
          validation_data =(xtest,ytest))




Train on 2700 samples, validate on 300 samples
Epoch 1/30





2700/2700 [==============================] - 49s 18ms/step - loss: 1.9731 - acc: 0.0112 - val_loss: 1.9412 - val_acc: 0.0116
Epoch 2/30
2700/2700 [==============================] - 37s 14ms/step - loss: 1.8699 - acc: 0.0116 - val_loss: 1.6535 - val_acc: 0.0116
Epoch 3/30
2700/2700 [==============================] - 37s 14ms/step - loss: 1.6081 - acc: 0.0125 - val_loss: 1.5393 - val_acc: 0.0135
Epoch 4/30
2700/2700 [==============================] - 37s 14ms/step - loss: 1.5382 - acc: 0.0135 - val_loss: 1.4956 - val_acc: 0.0136
Epoch 5/30
2700/2700 [==============================] - 38s 14ms/step - loss: 1.5033 - acc: 0.0145 - val_loss: 1.4726 - val_acc: 0.0144
Epoch 6/30
2700/2700 [==============================] - 38s 14ms/step - loss: 1.4817 - acc: 0.0160 - val_loss: 1.4562 - val_acc: 0.0248
Epoch 7/30
2700/2700 [==============================] - 39s 14ms/step - loss: 1.4659 - acc: 0.0172 - val_loss: 1.4399 - val_acc: 

KeyboardInterrupt: ignored

In [0]:
def calculate_bleu_score(actual_string, predicted_string):
  actual_string = copy.deepcopy(actual_string)
  predicted_string = copy.deepcopy(predicted_string)
  reference = re.split("\s",actual_string.strip())
  candidate = re.split("\s",predicted_string.strip())
  smoothie = SmoothingFunction().method4
  score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
  return score

In [0]:
def decode_input_sentence(arr,inverse_mapping,skip=False):
  line = ""
  for i in range(len(arr)):
    if arr[i] == 0.0:
      break
    if i == 0 and skip == True:
      continue
    if i == len(arr)-1 and skip == True:
      continue  
    line += inverse_mapping[arr[i]]+" "
    
  return line

In [0]:
def decode_model_output(model_output):
  # print(model_output)
  line = ""
  for i in range(len(model_output)):
    max_index = np.argmax(model_output[i, :])
    word = dest_index_to_word[max_index+1]
    if word == "<EOS>":
      # print(i)
      break
    line += word+" "
  return line

In [0]:

yhat = model.predict(xtest, verbose=0)
bleu_score = 0.0
for i in range(len(yhat)):
  src_input_sentence = decode_input_sentence(xtest[i,:],src_index_to_word)
  print(ytestp[i,:])
  dest_input_sentence = decode_input_sentence(ytestp[i,:],dest_index_to_word,True)
  output_sentence = decode_model_output(yhat[i,:])
  print("Actual Source: ",src_input_sentence)
  print("Actual Target: ",dest_input_sentence)
  print("predicted Target: ",output_sentence)
  curr_bs = calculate_bleu_score(dest_input_sentence,output_sentence)
  bleu_score += curr_bs
  print("Current BLEU Score: ",curr_bs)
  # break
print("Overall BLEU Score: ", bleu_score/len(yhat))